In [ ]:
import numpy as np
from PIL import Image
import tensorflow as tf
import pandas
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

In [ ]:
def convert_color_gray(filename,gray_string):
    img = Image.open('apple_train/' + filename).convert('LA')
    img.save('apple_gray/' + gray_string)
    return img.size


In [ ]:
def convert_gray_matrix(gray_string):
    img = Image.open('apple_gray/'+gray_string).convert('L')
    img_as_np = np.asarray(img.getdata()).reshape(img.size[1], img.size[0], -1)
    mat = img_as_np
    return mat

In [ ]:
def write_input_csv(grayMatrix,size,filename):
    g=[]
    for i in range(9):
        g.append(0)
    h,w = size
    with open(filename, "a") as res:
        for i in range(h):
                for j in range(w):
                    if(i - 1 == -1 or j - 1 == -1 or i + 1 == h or j + 1 == w):
                        inp_str = ""
                        for k in range(9):
                            inp_str += "0"+","
                        res.write(inp_str[:-1])
                        res.write("\n")
                    else:
                        g[0] = grayMatrix[i - 1][j - 1]
                        g[1] = g2 = grayMatrix[i - 1][j]
                        g[2] = grayMatrix[i - 1][j + 1]
                        g[3] = g4 = grayMatrix[i][j - 1]
                        g[4] = grayMatrix[i][j]
                        g[5] = grayMatrix[i][j + 1]
                        g[6]= grayMatrix[i + 1][j - 1]
                        g[7] = grayMatrix[i + 1][j]
                        g[8] = grayMatrix[i + 1][j + 1]
                        str = ""
                        for k in range(9):
                            str += np.array2string(g[k][0])+","
                        res.write(str[:-1])
                        res.write("\n")


In [ ]:
def write_ouput_csv(filename):
    image = Image.open('apple_train/'+filename)
    width,height = image.size
    pixelMatrix = np.asarray(list(image.getdata()),dtype='i,i,i')
    pixelMatrix =  np.reshape(pixelMatrix,(height*width,1))
    h, w = pixelMatrix.shape
    with open("output_final_apple.csv", "a") as resultsFile:
        for i in range(h):
            for j in range(w):
                out_string = ""
                for k in range(3):
                    out_string = out_string +  np.array2string(pixelMatrix[i][j][k]) +","                    
                resultsFile.write(out_string[:-1])
                resultsFile.write("\n")

### Generate Dataset

In [ ]:
if os.path.isfile('input.csv'):
        os.remove('input.csv')

if os.path.isfile('output_final_apple.csv'):
        os.remove('output_final_apple.csv')
        
if os.path.isfile('input_test_new.csv'):
        os.remove('input_test_new.csv')

In [ ]:
for filename in os.listdir('apple_train/'):
    gray_string = filename[:-4]+"_grayscale.png"
    size = convert_color_gray(filename,gray_string)
    mat = convert_gray_matrix(gray_string)
    write_input_csv(mat,size,'input.csv')
    write_ouput_csv(filename)

In [ ]:
file = open("input.csv")
numline = len(file.readlines())
print (numline)

### Load Dataset

In [ ]:
dataframe = pandas.read_csv("input.csv",sep=",", header=None)
dataframe2 = pandas.read_csv("output_final_apple.csv",sep=",", header=None)

X = dataframe.values
Y = dataframe2.values
R = dataframe2.values[:,0]
G = dataframe2.values[:,1]
B = dataframe2.values[:,2]


### Neural Network Model
The Network consists of total 5 layers


In [ ]:
Input_1= Input(shape=(9, ))

x = Dense(9, activation='linear')(Input_1)
x = Dense(5, activation='relu')(x)
x = Dense(4, activation='linear')(x)
x = Dense(5, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)

r = Dense(1,  activation='linear')(x)
g = Dense(1,  activation='linear')(x)
b = Dense(1,  activation='linear')(x)

model = Model(inputs=Input_1, outputs=[r,g,b])
model.compile(optimizer = "rmsprop", loss = 'mse', metrics = ['accuracy'] )

### Training Model 
Training model using 100 epochs and batch size of 100 and validation data split of 33%

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss', patience = 100),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

In [ ]:
history = model.fit(X, [R,G,B], epochs=1000, batch_size = 100, validation_split=0.33,callbacks=callbacks )

### Evaluation of model

In [ ]:
model.evaluate(X,[R,G,B])

### Testing Model

In [ ]:
img = Image.open('apple_test.jpg').convert('LA')
img.save('apple_test_gray.png')
img = Image.open('apple_test_gray.png').convert('L')
img_as_np = np.asarray(img.getdata()).reshape(img.size[1], img.size[0], -1)
mat = img_as_np
write_input_csv(mat,img.size,'input_test_new.csv')

In [ ]:
dataframe = pandas.read_csv("input_test_new.csv",sep=",", header=None)
dataframe[:2]
pred = model.predict(dataframe)

### Testing on apple image

In [ ]:
height = mat.shape[0]
with open('neural_color_output_final_apple.csv', "w") as res:
        for i in range(len(pred[0])):
                res.write(np.array2string(pred[0][i][0]) + "," +np.array2string(pred[1][i][0]) + "," +np.array2string(pred[2][i][0]) + "\n")
colors = np.genfromtxt('neural_color_output_final_apple.csv', dtype='i,i,i', delimiter=",")
colors = np.reshape(colors,(height,height))
colorMatrix = np.zeros((height,height,3),'uint8')
for i in range(height):
    for j in range(height):
        colorMatrix[i][j] = tuple((colors[i][j][0],colors[i][j][1],colors[i][j][2]))
image = Image.fromarray(colorMatrix)
image.save("output.jpeg")
img=mpimg.imread("apple_test.jpg")
plt.title('Orignal')
imgplot = plt.imshow(img)
plt.show()
img=mpimg.imread("output.jpeg")
plt.title('Colorized')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
model.metrics_names

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# img = Image.open('ava_test.jpg').convert('LA')
# img.save('ava_test_gray.png')
# img = Image.open('ava_test_gray.png').convert('L')
# img_as_np = np.asarray(img.getdata()).reshape(img.size[1], img.size[0], -1)
# mat = img_as_np
# write_input_csv(mat,img.size,'ava_test_new2.csv')
# dataframe = pandas.read_csv("ava_test_new2.csv",sep=",", header=None)
# dataframe[:2]
# pred = model.predict(dataframe)
# height = mat.shape[0]
# with open('neural.csv', "w") as res:
#         for i in range(len(pred[0])):
#                 res.write(np.array2string(pred[0][i][0]) + "," +np.array2string(pred[1][i][0]) + "," +np.array2string(pred[2][i][0]) + "\n")
# colors = np.genfromtxt('neural.csv', dtype='i,i,i', delimiter=",")
# colors = np.reshape(colors,(height,height))
# colorMatrix = np.zeros((height,height,3),'uint8')
# for i in range(height):
#     for j in range(height):
#         colorMatrix[i][j] = tuple((colors[i][j][0],colors[i][j][1],colors[i][j][2]))
# image = Image.fromarray(colorMatrix)
# image.save("your_file_final_ava.jpeg")
# img=mpimg.imread("ava_test.jpg")
# plt.title('Orignal')
# imgplot = plt.imshow(img)
# plt.show()
# img=mpimg.imread("your_file_final_ava.jpeg")
# plt.title('Colorized')
# imgplot = plt.imshow(img)
# plt.show()